In [1]:
from grand import ncmc_sampler as ns
from grand import lig_utils as lu

import openmm
import openmm.unit
import openff.toolkit
import pdbfixer

import numpy as np
import mdtraj as md
from scipy.spatial import Voronoi, Delaunay

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
datadir = '../../examples/lysozyme/'
# Load protein and ligand
prot = openmm.app.PDBFile(f'{datadir}/apo.pdb')
lig = openff.toolkit.Molecule.from_smiles("c1ccccc1O")
lig.generate_conformers(n_conformers=1000)
lig_top = lig.to_topology().to_openmm()
lig_pos = lig.conformers[0].to_openmm()

# Load forcefield
ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml',f'{datadir}/phenol_openff.xml')

In [3]:
# Add one ligand to the system
sys_top, sys_pos, ghosts = lu.add_ghosts(prot.topology, prot.positions, lig_top, lig_pos, n=1, output=f'{datadir}/ncmc.pdb')

100%|██████████| 1/1 [00:00<00:00, 40.42it/s]


In [4]:
# Load pdb with protein and ligand
pdb = openmm.app.PDBFile(f'{datadir}/ncmc.pdb')

In [5]:
# Solvate the system
#modeller = openmm.app.Modeller(pdb.topology, pdb.positions)
#modeller.addSolvent(forcefield=ff,padding=1*openmm.unit.nanometer)
#openmm.app.PDBFile.writeFile(modeller.topology, modeller.positions, f'{datadir}/ncmc_sol.pdb')

In [6]:
# Compute Voronoi vertices
traj = md.load(f'{datadir}/apo.pdb')
traj = traj.atom_slice(traj.top.select('name CA'))
vor = Voronoi(traj.xyz[0])
vor = vor.vertices[np.prod((vor.vertices > traj.xyz.min()) & (vor.vertices < traj.xyz.max()),axis=1).astype(bool)]

In [7]:
frag_res_ids = []
frag_atom_ids = []
alchemical_atom_ids = []
for resid, residue in enumerate(pdb.topology.residues()):
    if residue.name == "UNK":
        frag_res_ids.append(resid)
        for atom in residue.atoms():
            frag_atom_ids.append(atom.index)
    for atom in residue.atoms():
        alchemical_atom_ids.append(atom.index)

In [8]:
pdb_system = ff.createSystem(pdb.topology)

In [9]:
reporter = openmm.app.DCDReporter(f"{datadir}/ncmc_test.dcd", reportInterval=10000)

In [10]:
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics': x,
                             'lambda_electrostatics': x,
                             }
# Non-linear lambda
#DEFAULT_ALCHEMICAL_FUNCTIONS = {
#                             'lambda_sterics': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 1.0)",
#                             'lambda_electrostatics': f"select(step({x} - 0.5), 0.0 , 2.0 * (0.5 - {x}))",
#                             }

In [11]:
# Visualize Voronoi vertices
"""
f = open(f'{datadir}/voronoi_vertices.pdb','w')
f.write('REMARK\n')
for i, vertex in enumerate(vor*10):
    f.write(f'ATOM  {i+1:>5}  H   VOR A   1    {vertex[0]:8.3f}{vertex[1]:8.3f}{vertex[2]:8.3f}  1.00  0.00           H  \n')
f.close()
"""

"\nf = open(f'{datadir}/voronoi_vertices.pdb','w')\nf.write('REMARK\n')\nfor i, vertex in enumerate(vor*10):\n    f.write(f'ATOM  {i+1:>5}  H   VOR A   1    {vertex[0]:8.3f}{vertex[1]:8.3f}{vertex[2]:8.3f}  1.00  0.00           H  \n')\nf.close()\n"

In [12]:
# Initialize Sampler
sampler = ns.BaseNCMCSampler(pdb_system,pdb.topology,pdb.positions,reporter=reporter,alchemical_atoms=alchemical_atom_ids,
                             alchemical_functions=DEFAULT_ALCHEMICAL_FUNCTIONS,periodic=False, nsteps_neq=10,nsteps_eq=10000, frag_atoms=frag_atom_ids, voronoi_vertices=[openmm.Vec3(0.4961, 1.4597, 2.3242)])

In [13]:
# Run sampler
sampler.ncmc_move(5)

100%|██████████| 5/5 [00:00<00:00, 18.74it/s]


In [14]:
# Print algorithm of integrator
sampler.nc_integrator.pretty_print()

step      0 : if(step = 0):
step      1 :    constrain positions
step      2 :    constrain velocities
step      3 :    protocol_work <- 0.0
step      4 :    lambda <- 0
step      5 :    protocol_work <- 0
step      6 :    step <- 0
step      7 :    lambda_step <- 0
step      8 :    lambda_sterics <- lambda
step      9 :    lambda_electrostatics <- lambda
step     10 : end
step     11 : if(step >= 0):
step     12 :    if(step < n_steps_per_cycle):
step     13 :       allow forces to update the context state
step     14 :       if(has_kT_changed = 1):
step     15 :          sigma <- sqrt(kT/m)
step     16 :          has_kT_changed <- 0
step     17 :       end
step     18 :       v <- v + (dt / 2) * f / m
step     19 :       constrain velocities
step     20 :       x <- x + ((dt / 2) * v)
step     21 :       x1 <- x
step     22 :       constrain positions
step     23 :       v <- v + ((x - x1) / (dt / 2))
step     24 :       constrain velocities
step     25 :       Eold <- energy
step   